# Feature Engineering Pipeline - Black Friday Sales

**Cel:** Stworzenie zaawansowanych cech dla modeli Machine Learning

**Pipeline:**
1. Ładowanie danych surowych
2. Feature Engineering:
   - Agregacje per użytkownik
   - Agregacje per produkt
   - RFM Features
   - Categorical Encoding
   - Interakcje cech
3. Zapis do Delta Lake

---

In [1]:
# Imports
import sys
sys.path.append('../')

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, sum as _sum, avg, min as _min, max as _max,
    countDistinct, stddev, when, lit, dense_rank, percent_rank,
    broadcast, concat_ws, row_number
)
from pyspark.sql.window import Window
from pyspark.ml.feature import (
    StringIndexer, OneHotEncoder, VectorAssembler,
    StandardScaler, MinMaxScaler
)
from pyspark.ml import Pipeline

from config.spark_config import SparkConfig

import warnings
warnings.filterwarnings('ignore')

## 1. Inicjalizacja Spark Session

In [2]:
# Start Spark Session
spark = SparkConfig.get_spark_session("BlackFriday-FeatureEngineering")

print("\n✓ Spark session ready!")
print(f"Spark UI: http://localhost:4040")

✓ Windows: HADOOP_HOME set to C:\Users\usate\PycharmProjects\BlackFriday\hadoop
  Note: If you encounter permission errors, download winutils.exe
  from https://github.com/steveloughran/winutils and place in hadoop/bin/
Spark Session Created: BlackFriday-FeatureEngineering
Spark Version: 3.5.3
Master: local[*]
App Name: BlackFriday-FeatureEngineering

✓ Spark session ready!
Spark UI: http://localhost:4040


## 2. Ładowanie Danych

In [3]:
# Load raw data
df_raw = spark.read.csv(
    "../data/raw/train.csv",
    header=True,
    inferSchema=True
)

print(f"\nDataset shape: {df_raw.count():,} rows x {len(df_raw.columns)} columns")
df_raw.printSchema()


Dataset shape: 550,068 rows x 12 columns
root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [4]:
# Show sample
df_raw.show(5, truncate=False)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age |Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001|P00069042 |F     |0-17|10        |A            |2                         |0             |3                 |NULL              |NULL              |8370    |
|1000001|P00248942 |F     |0-17|10        |A            |2                         |0             |1                 |6                 |14                |15200   |
|1000001|P00087842 |F     |0-17|10        |A            |2                         |0             |12                |NULL              |NULL              |1422    |
|100

## 3. Data Quality & Cleansing

In [5]:
# Check for duplicates
total_rows = df_raw.count()
unique_rows = df_raw.dropDuplicates().count()

print(f"Total rows: {total_rows:,}")
print(f"Unique rows: {unique_rows:,}")
print(f"Duplicates: {total_rows - unique_rows:,}")

Total rows: 550,068
Unique rows: 550,068
Duplicates: 0


In [6]:
# Check missing values
from pyspark.sql.functions import isnan, isnull

missing_counts = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df_raw.columns
])

print("\nMissing values per column:")
missing_counts.show()


Missing values per column:
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|            173638|            383247|       0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [7]:
# Clean data
df_clean = df_raw.dropDuplicates()

# Fill missing Product_Category_2 and Product_Category_3 with 0 (indicating no secondary category)
df_clean = df_clean.fillna({
    'Product_Category_2': 0,
    'Product_Category_3': 0
})

print(f"\n✓ Cleaned data: {df_clean.count():,} rows")


✓ Cleaned data: 550,068 rows


## 4. Feature Engineering - User-Level Aggregations

In [8]:
# User aggregations
user_features = df_clean.groupBy("User_ID").agg(
    # Purchase statistics
    count("*").alias("user_purchase_count"),
    _sum("Purchase").alias("user_total_spent"),
    avg("Purchase").alias("user_avg_purchase"),
    _min("Purchase").alias("user_min_purchase"),
    _max("Purchase").alias("user_max_purchase"),
    stddev("Purchase").alias("user_std_purchase"),
    
    # Product diversity
    countDistinct("Product_ID").alias("user_unique_products"),
    countDistinct("Product_Category_1").alias("user_unique_categories"),
    
    # Categorical modes (most frequent)
    # We'll take the first value as a proxy for mode
)

# Add derived features
user_features = user_features.withColumn(
    "user_purchase_range",
    col("user_max_purchase") - col("user_min_purchase")
)

user_features = user_features.withColumn(
    "user_avg_products_per_transaction",
    col("user_unique_products") / col("user_purchase_count")
)

print(f"\nUser features created: {user_features.count():,} users")
user_features.show(5)


User features created: 5,891 users
+-------+-------------------+----------------+-----------------+-----------------+-----------------+-----------------+--------------------+----------------------+-------------------+---------------------------------+
|User_ID|user_purchase_count|user_total_spent|user_avg_purchase|user_min_purchase|user_max_purchase|user_std_purchase|user_unique_products|user_unique_categories|user_purchase_range|user_avg_products_per_transaction|
+-------+-------------------+----------------+-----------------+-----------------+-----------------+-----------------+--------------------+----------------------+-------------------+---------------------------------+
|1003031|                290|         2585988|           8917.2|               49|            20855|3957.360091540527|                 290|                    14|              20806|                              1.0|
|1000190|                 81|          778581|9612.111111111111|               24|            23

## 5. Feature Engineering - Product-Level Aggregations

In [9]:
# Product aggregations
product_features = df_clean.groupBy("Product_ID").agg(
    count("*").alias("product_purchase_count"),
    countDistinct("User_ID").alias("product_unique_users"),
    _sum("Purchase").alias("product_total_revenue"),
    avg("Purchase").alias("product_avg_price"),
    _min("Purchase").alias("product_min_price"),
    _max("Purchase").alias("product_max_price"),
    stddev("Purchase").alias("product_std_price")
)

# Product popularity ranking
window_spec = Window.orderBy(col("product_purchase_count").desc())
product_features = product_features.withColumn(
    "product_popularity_rank",
    dense_rank().over(window_spec)
)

# Normalize popularity to 0-1 scale
product_features = product_features.withColumn(
    "product_popularity_score",
    percent_rank().over(window_spec)
)

print(f"\nProduct features created: {product_features.count():,} products")
product_features.show(5)


Product features created: 3,631 products
+----------+----------------------+--------------------+---------------------+------------------+-----------------+-----------------+------------------+-----------------------+------------------------+
|Product_ID|product_purchase_count|product_unique_users|product_total_revenue| product_avg_price|product_min_price|product_max_price| product_std_price|product_popularity_rank|product_popularity_score|
+----------+----------------------+--------------------+---------------------+------------------+-----------------+-----------------+------------------+-----------------------+------------------------+
| P00265242|                  1880|                1880|             14165515| 7534.848404255319|             1720|             8907|1683.9859563927332|                      1|                     0.0|
| P00025442|                  1615|                1615|             27995166| 17334.46811145511|             3961|            19707|2955.609692490171

## 6. Feature Engineering - Category-Level Aggregations

In [10]:
# Category aggregations
category_features = df_clean.groupBy("Product_Category_1").agg(
    count("*").alias("category_purchase_count"),
    avg("Purchase").alias("category_avg_price"),
    _sum("Purchase").alias("category_total_revenue")
)

print(f"\nCategory features: {category_features.count()} categories")
category_features.orderBy(col("category_total_revenue").desc()).show(10)


Category features: 20 categories
+------------------+-----------------------+------------------+----------------------+
|Product_Category_1|category_purchase_count|category_avg_price|category_total_revenue|
+------------------+-----------------------+------------------+----------------------+
|                 1|                 140378|13606.218595506418|            1910013754|
|                 5|                 150933| 6240.088178198273|             941835229|
|                 8|                 113925| 7498.958077682686|             854318799|
|                 6|                  20466|15838.478549789896|             324150302|
|                 2|                  23864| 11251.93538384177|             268516186|
|                 3|                  20213|10096.705733933608|             204084713|
|                16|                   9828|14766.037037037036|             145120612|
|                11|                  24287| 4685.268456375839|             113791115|
|        

## 7. RFM Features (Recency, Frequency, Monetary)

In [11]:
# Create RFM features
# Note: Dataset doesn't have timestamps, so we'll create proxy RFM based on transaction ordering

# Frequency: Already have user_purchase_count
# Monetary: Already have user_total_spent
# Recency: Use row_number as proxy for time (last purchase)

window_user = Window.partitionBy("User_ID").orderBy(col("Purchase").desc())

df_with_recency = df_clean.withColumn(
    "transaction_order",
    row_number().over(window_user)
)

# Get recency (1 = most recent for each user)
recency_features = df_with_recency.filter(col("transaction_order") == 1).select(
    "User_ID",
    col("Purchase").alias("last_purchase_amount")
)

# Join recency with user features to create full RFM
rfm_features = user_features.join(
    recency_features,
    on="User_ID",
    how="left"
)

# RFM Scoring (quintiles)
rfm_features = rfm_features.withColumn(
    "rfm_frequency_score",
    percent_rank().over(Window.orderBy(col("user_purchase_count")))
)

rfm_features = rfm_features.withColumn(
    "rfm_monetary_score",
    percent_rank().over(Window.orderBy(col("user_total_spent")))
)

# Combined RFM score
rfm_features = rfm_features.withColumn(
    "rfm_score",
    (col("rfm_frequency_score") + col("rfm_monetary_score")) / 2
)

print("\nRFM Features:")
rfm_features.select(
    "User_ID", "user_purchase_count", "user_total_spent",
    "rfm_frequency_score", "rfm_monetary_score", "rfm_score"
).show(10)


RFM Features:
+-------+-------------------+----------------+--------------------+--------------------+--------------------+
|User_ID|user_purchase_count|user_total_spent| rfm_frequency_score|  rfm_monetary_score|           rfm_score|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+
|1004464|                  9|           46681|0.003056027164685...|                 0.0|0.001528013582342...|
|1000094|                  7|           49288|1.697792869269949...|1.697792869269949...|1.697792869269949...|
|1003883|                  9|           49349|0.003056027164685...|3.395585738539898...|0.001697792869269949|
|1005117|                  9|           49668|0.003056027164685...|5.093378607809847E-4|0.001782682512733...|
|1004991|                  7|           52371|1.697792869269949...|6.791171477079797E-4|4.244482173174872...|
|1005944|                 15|           53996| 0.06791171477079797|8.488964346349745E-4| 0.03438030560271

## 8. Join All Features

In [12]:
# Join user features back to main dataset
df_enriched = df_clean.join(
    broadcast(user_features),
    on="User_ID",
    how="left"
)

# Join product features
df_enriched = df_enriched.join(
    broadcast(product_features),
    on="Product_ID",
    how="left"
)

# Join category features
df_enriched = df_enriched.join(
    broadcast(category_features),
    on="Product_Category_1",
    how="left"
)

# Join RFM features
df_enriched = df_enriched.join(
    broadcast(rfm_features.select(
        "User_ID", "rfm_frequency_score", "rfm_monetary_score", "rfm_score"
    )),
    on="User_ID",
    how="left"
)

print(f"\n✓ Enriched dataset: {df_enriched.count():,} rows x {len(df_enriched.columns)} columns")
print("\nNew features added:")
print([c for c in df_enriched.columns if c not in df_clean.columns])


✓ Enriched dataset: 550,068 rows x 37 columns

New features added:
['user_purchase_count', 'user_total_spent', 'user_avg_purchase', 'user_min_purchase', 'user_max_purchase', 'user_std_purchase', 'user_unique_products', 'user_unique_categories', 'user_purchase_range', 'user_avg_products_per_transaction', 'product_purchase_count', 'product_unique_users', 'product_total_revenue', 'product_avg_price', 'product_min_price', 'product_max_price', 'product_std_price', 'product_popularity_rank', 'product_popularity_score', 'category_purchase_count', 'category_avg_price', 'category_total_revenue', 'rfm_frequency_score', 'rfm_monetary_score', 'rfm_score']


## 9. Interaction Features

In [13]:
# Create interaction features
df_enriched = df_enriched.withColumn(
    "purchase_vs_user_avg_ratio",
    col("Purchase") / col("user_avg_purchase")
)

df_enriched = df_enriched.withColumn(
    "purchase_vs_product_avg_ratio",
    col("Purchase") / col("product_avg_price")
)

df_enriched = df_enriched.withColumn(
    "purchase_vs_category_avg_ratio",
    col("Purchase") / col("category_avg_price")
)

# Is this purchase above user's average?
df_enriched = df_enriched.withColumn(
    "is_above_user_avg",
    when(col("Purchase") > col("user_avg_purchase"), 1).otherwise(0)
)

# High value customer flag (top 20%)
df_enriched = df_enriched.withColumn(
    "is_high_value_customer",
    when(col("rfm_score") >= 0.8, 1).otherwise(0)
)

print("\n✓ Interaction features created")


✓ Interaction features created


## 10. Categorical Encoding

In [14]:
# Define categorical columns
categorical_cols = [
    "Gender",
    "Age",
    "City_Category",
    "Stay_In_Current_City_Years"
]

# String Indexing
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep")
    for col in categorical_cols
]

# One-Hot Encoding
encoders = [
    OneHotEncoder(inputCol=col + "_index", outputCol=col + "_vec", dropLast=False)
    for col in categorical_cols
]

# Create pipeline
encoding_pipeline = Pipeline(stages=indexers + encoders)

# Fit and transform
encoding_model = encoding_pipeline.fit(df_enriched)
df_encoded = encoding_model.transform(df_enriched)

print("\n✓ Categorical encoding completed")
print(f"Total columns after encoding: {len(df_encoded.columns)}")


✓ Categorical encoding completed
Total columns after encoding: 50


## 11. Feature Selection for ML

In [15]:
# Select features for ML models
feature_cols = [
    # Categorical (indexed)
    "Gender_index", "Age_index", "City_Category_index",
    "Stay_In_Current_City_Years_index",
    
    # Numerical demographics
    "Occupation", "Marital_Status",
    
    # Product categories
    "Product_Category_1", "Product_Category_2", "Product_Category_3",
    
    # User features
    "user_purchase_count", "user_total_spent", "user_avg_purchase",
    "user_std_purchase", "user_unique_products", "user_unique_categories",
    
    # Product features
    "product_purchase_count", "product_unique_users", "product_avg_price",
    "product_popularity_score",
    
    # Category features
    "category_avg_price",
    
    # RFM features
    "rfm_frequency_score", "rfm_monetary_score", "rfm_score",
    
    # Interaction features
    "purchase_vs_user_avg_ratio", "purchase_vs_product_avg_ratio",
    "is_above_user_avg", "is_high_value_customer"
]

# Fill any remaining nulls with 0
df_encoded = df_encoded.fillna(0, subset=feature_cols)

# Vector Assembler
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

df_features = assembler.transform(df_encoded)

print(f"\n✓ Feature vector assembled with {len(feature_cols)} features")


✓ Feature vector assembled with 27 features


## 12. Feature Scaling

In [16]:
# Standard Scaler
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withMean=False,  # Sparse vectors don't support centering
    withStd=True
)

scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

print("\n✓ Features scaled")


✓ Features scaled


## 13. Final Feature Set Summary

In [17]:
# Select final columns for ML
final_cols = [
    # IDs
    "User_ID", "Product_ID",
    
    # Target
    "Purchase",
    
    # Original features
    "Gender", "Age", "Occupation", "City_Category",
    "Stay_In_Current_City_Years", "Marital_Status",
    "Product_Category_1", "Product_Category_2", "Product_Category_3",
    
    # Engineered features (all)
] + [c for c in df_scaled.columns if c.startswith(("user_", "product_", "category_", "rfm_", "is_", "purchase_vs"))] + [
    # ML-ready features
    "features", "scaled_features"
]

df_final = df_scaled.select(*final_cols)

print("\n" + "="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)
print(f"Final dataset: {df_final.count():,} rows x {len(df_final.columns)} columns")
print(f"\nTotal features in vector: {len(feature_cols)}")
print("\nFeature categories:")
print(f"  - User features: {len([c for c in feature_cols if c.startswith('user_')])}")
print(f"  - Product features: {len([c for c in feature_cols if c.startswith('product_')])}")
print(f"  - Category features: {len([c for c in feature_cols if c.startswith('category_')])}")
print(f"  - RFM features: {len([c for c in feature_cols if c.startswith('rfm_')])}")
print(f"  - Interaction features: {len([c for c in feature_cols if 'ratio' in c or 'is_' in c])}")
print(f"  - Demographics: {len([c for c in feature_cols if c in categorical_cols or c in ['Occupation', 'Marital_Status']])}")


FEATURE ENGINEERING COMPLETE
Final dataset: 550,068 rows x 44 columns

Total features in vector: 27

Feature categories:
  - User features: 6
  - Product features: 4
  - Category features: 1
  - RFM features: 3
  - Interaction features: 4
  - Demographics: 2


In [18]:
# Show sample
df_final.select(
    "User_ID", "Product_ID", "Purchase",
    "user_avg_purchase", "product_popularity_score", "rfm_score"
).show(10)

+-------+----------+--------+------------------+------------------------+------------------+
|User_ID|Product_ID|Purchase| user_avg_purchase|product_popularity_score|         rfm_score|
+-------+----------+--------+------------------+------------------------+------------------+
|1000125| P00182142|   19064|14017.444444444445|    0.014600550964187328|0.4241086587436333|
|1000146| P00088242|   12053| 9484.264705882353|      0.2793388429752066|0.9141765704584041|
|1000163| P00270942|   12017| 6028.527675276753|    0.007988980716253443|0.8886247877758914|
|1000170| P00111142|   19696| 12508.53488372093|    0.009366391184573003|0.4659592529711375|
|1000438| P00254242|    8868|  8671.30745341615|     0.06666666666666667| 0.951528013582343|
|1000541| P00023342|    5473| 8784.018018018018|     0.43526170798898073|0.7220713073005094|
|1000588| P00116942|   16549| 9311.297709923665|      0.2862258953168044|0.9272495755517827|
|1000620| P00127942|   11734|  7990.36170212766|     0.062534435261707

## 14. Save to Delta Lake

In [19]:
# Save to Delta Lake
delta_path = "../data/processed/delta/features"

df_final.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("City_Category", "Age") \
    .save(delta_path)

print(f"\n✓ Data saved to Delta Lake: {delta_path}")


✓ Data saved to Delta Lake: ../data/processed/delta/features


In [20]:
# Save individual feature sets for specific use cases

# 1. User-level features (for segmentation)
rfm_features.write \
    .format("delta") \
    .mode("overwrite") \
    .save("../data/processed/delta/user_features")

print("✓ User features saved")

# 2. Product-level features (for recommendations)
product_features.write \
    .format("delta") \
    .mode("overwrite") \
    .save("../data/processed/delta/product_features")

print("✓ Product features saved")

# 3. Category features
category_features.write \
    .format("delta") \
    .mode("overwrite") \
    .save("../data/processed/delta/category_features")

print("✓ Category features saved")

print("\n" + "="*60)
print("✅ ALL DATA SAVED TO DELTA LAKE")
print("="*60)
print("\nSaved locations:")
print("  - Main features: data/processed/delta/features/")
print("  - User features: data/processed/delta/user_features/")
print("  - Product features: data/processed/delta/product_features/")
print("  - Category features: data/processed/delta/category_features/")

print("\n📖 How to load:")
print("""
# Load from Delta Lake
df = spark.read.format("delta").load("../data/processed/delta/features")
users = spark.read.format("delta").load("../data/processed/delta/user_features")
products = spark.read.format("delta").load("../data/processed/delta/product_features")
""")
print("="*60)

✓ User features saved
✓ Product features saved
✓ Category features saved

✅ ALL DATA SAVED TO DELTA LAKE

Saved locations:
  - Main features: data/processed/delta/features/
  - User features: data/processed/delta/user_features/
  - Product features: data/processed/delta/product_features/
  - Category features: data/processed/delta/category_features/

📖 How to load:

# Load from Delta Lake
df = spark.read.format("delta").load("../data/processed/delta/features")
users = spark.read.format("delta").load("../data/processed/delta/user_features")
products = spark.read.format("delta").load("../data/processed/delta/product_features")



## 15. Validation & Statistics

In [21]:
# Feature statistics
print("\n" + "="*60)
print("FEATURE STATISTICS")
print("="*60)

# Select numerical features for stats
numerical_features = [
    "Purchase",
    "user_avg_purchase", "user_purchase_count", "user_total_spent",
    "product_popularity_score", "product_avg_price",
    "rfm_score"
]

df_final.select(numerical_features).describe().show()


FEATURE STATISTICS
+-------+-----------------+------------------+-------------------+------------------+------------------------+------------------+--------------------+
|summary|         Purchase| user_avg_purchase|user_purchase_count|  user_total_spent|product_popularity_score| product_avg_price|           rfm_score|
+-------+-----------------+------------------+-------------------+------------------+------------------------+------------------+--------------------+
|  count|           550068|            550068|             550068|            550068|                  550068|            550068|              550068|
|   mean|9263.968712959126| 9263.968712959091| 216.40340103405399|1925314.2293225564|     0.18545974861790282| 9263.968712959126|  0.7600518627960817|
| stddev|5023.065393820586|1619.5105670471598| 175.03115005145187|1489900.5977689005|      0.1842330766840676| 4267.891837442049| 0.23262711582228426|
|    min|               12| 2318.733333333333|                  6|        

In [22]:
# Check for nulls in final dataset
null_counts = df_final.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df_final.columns[:20]  # First 20 cols
])

print("\nNull counts in final dataset (first 20 columns):")
null_counts.show()


Null counts in final dataset (first 20 columns):
+-------+----------+--------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+-------------------+----------------+-----------------+-----------------+-----------------+-----------------+--------------------+----------------------+
|User_ID|Product_ID|Purchase|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|user_purchase_count|user_total_spent|user_avg_purchase|user_min_purchase|user_max_purchase|user_std_purchase|user_unique_products|user_unique_categories|
+-------+----------+--------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+-------------------+----------------+-----------------+-----------------+-----------------+-----------------+--------------------+--------------------

In [23]:
# Feature correlation with target (Purchase)
from pyspark.ml.stat import Correlation

# Correlation matrix
correlation_matrix = Correlation.corr(df_final, "scaled_features").head()

print("\n✓ Correlation matrix computed")
print("(Can be used for feature selection)")


✓ Correlation matrix computed
(Can be used for feature selection)


## 16. Summary & Next Steps

In [24]:
print("\n" + "="*60)
print("✅ FEATURE ENGINEERING PIPELINE COMPLETE")
print("="*60)

print("\n📊 Created Features:")
print("  ✓ User-level aggregations (purchase patterns)")
print("  ✓ Product-level aggregations (popularity, pricing)")
print("  ✓ Category-level aggregations")
print("  ✓ RFM features (Frequency, Monetary)")
print("  ✓ Interaction features (ratios, flags)")
print("  ✓ Categorical encoding (StringIndexer + OneHot)")
print("  ✓ Feature scaling (StandardScaler)")

print("\n💾 Saved Artifacts:")
print("  ✓ Main features dataset → data/processed/delta/features/")
print("  ✓ User features → data/processed/delta/user_features/")
print("  ✓ Product features → data/processed/delta/product_features/")
print("  ✓ Encoding pipeline → models/encoding_pipeline/")
print("  ✓ Scaler model → models/scaler_model/")

print("\n🎯 Next Steps:")
print("  1. Build ETL pipeline (automation)")
print("  2. Train ML models (regression, clustering, recommendation)")
print("  3. Model evaluation and tuning")
print("  4. Setup Kafka + Streaming pipeline")

print("\n" + "="*60)


✅ FEATURE ENGINEERING PIPELINE COMPLETE

📊 Created Features:
  ✓ User-level aggregations (purchase patterns)
  ✓ Product-level aggregations (popularity, pricing)
  ✓ Category-level aggregations
  ✓ RFM features (Frequency, Monetary)
  ✓ Interaction features (ratios, flags)
  ✓ Categorical encoding (StringIndexer + OneHot)
  ✓ Feature scaling (StandardScaler)

💾 Saved Artifacts:
  ✓ Main features dataset → data/processed/delta/features/
  ✓ User features → data/processed/delta/user_features/
  ✓ Product features → data/processed/delta/product_features/
  ✓ Encoding pipeline → models/encoding_pipeline/
  ✓ Scaler model → models/scaler_model/

🎯 Next Steps:
  1. Build ETL pipeline (automation)
  2. Train ML models (regression, clustering, recommendation)
  3. Model evaluation and tuning
  4. Setup Kafka + Streaming pipeline



In [25]:
# Stop Spark session
# spark.stop()
# print("\n✓ Spark session stopped")